In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM 

In [6]:
df = pd.read_csv('data/AI_Human.csv')
df.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [29]:
len(df)

487235

In [30]:
# Assuming df is your original dataframe
df_zero = df[df['generated'] == 0]
df_one = df[df['generated'] == 1]

# Sample 5000 rows from each dataframe
df_zero_sampled = df_zero.sample(5000, random_state=1)
df_one_sampled = df_one.sample(5000, random_state=1)

# Concatenate the two dataframes
df = pd.concat([df_zero_sampled, df_one_sampled])

In [31]:
# Train/test split
X = df.drop(columns=['generated'])
y = df['generated']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=45)

In [32]:
# Determining the max length (in words) of rows of the data
maxlen = df['text'].apply(lambda x: len(x.split())).max()
print(f'Max length: {maxlen}')

Max length: 1642


In [33]:
# Estimating the size of the dataset's vocabulary
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
vocab_size = len(tokenizer.word_index)

print(f'Estimated vocabulary size: {vocab_size}')

Estimated vocabulary size: 40378


In [36]:
# Setting the number of words to consider as features
max_features = vocab_size

# Converting to list
X_train = X_train['text'].tolist()
X_test = X_test['text'].tolist()

# Instantiating the tokenizer
tokenizer = Tokenizer(num_words=max_features, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Converting texts to sequences
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Padding the sequences
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the LSTM model
model = Sequential([
    Embedding(max_features, 32),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='rmsprop', # Using rmsprop as the optimizer as it's good for RNNs
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=10,
          batch_size=128,
          validation_split=0.2) # Setting validation split to monitor the model's performance and prevent overfitting.

Epoch 1/10
47/47 [==============================] - 24s 493ms/step - loss: 0.5511 - accuracy: 0.7748 - val_loss: 0.4592 - val_accuracy: 0.8973
Epoch 2/10
47/47 [==============================] - 23s 493ms/step - loss: 0.2948 - accuracy: 0.9175 - val_loss: 0.2216 - val_accuracy: 0.9453
Epoch 3/10
47/47 [==============================] - 22s 478ms/step - loss: 0.1757 - accuracy: 0.9558 - val_loss: 0.1657 - val_accuracy: 0.9593
Epoch 4/10
47/47 [==============================] - 22s 479ms/step - loss: 0.1219 - accuracy: 0.9690 - val_loss: 0.1145 - val_accuracy: 0.9720
Epoch 5/10
47/47 [==============================] - 22s 478ms/step - loss: 0.0721 - accuracy: 0.9850 - val_loss: 0.1915 - val_accuracy: 0.9453
Epoch 6/10
47/47 [==============================] - 23s 488ms/step - loss: 0.0631 - accuracy: 0.9853 - val_loss: 0.0866 - val_accuracy: 0.9787
Epoch 7/10
47/47 [==============================] - 22s 476ms/step - loss: 0.0455 - accuracy: 0.9895 - val_loss: 0.0865 - val_accuracy: 0.9753

In [38]:
# Evaluating the model on test data
results = model.evaluate(X_test, y_test)

79/79 [==============================] - 4s 45ms/step - loss: 0.0853 - accuracy: 0.9804


In [39]:
# Answers to the assignment.
# Note! Do not edit this cell, just run it after you complete the task. 

# TODO: Put the result of the model.evaluate() function in the results variable.
# TODO: set metrics=['accuracy'] to monitor in model.compile() to see how much of the neural network's predictions is correct at different stages of training.

print(f"Test Loss:{results[0]} Test Accuracy:{results[1]*100}%")

Test Loss:0.08526962995529175 Test Accuracy:98.0400025844574%
